## Resumo dos relatórios dos diversos Vendors (utilizando LLM)

In [44]:
import openai
openai.__version__

import os
from openai import AzureOpenAI
from configparser import ConfigParser, ExtendedInterpolation
import httpx
import numpy as np
import pandas as pd
import PyPDF2
from datetime import datetime

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('config-v1.x.ini', 'UTF-8')

http_client = httpx.Client(verify='petrobras-ca-root.pem')

client = AzureOpenAI(
    api_key=config['TST']['API_KEY_MODELOS_TEXTO'],
    api_version=config['OPENAI']['OPENAI_API_VERSION'],
    azure_endpoint=config['TST']['LITELLM_BASE_URL'],
    http_client=http_client
)

### Carrega Dados Básicos

In [45]:
# Set the correct path to the Excel file
excel_filename = 'C:/Users/ELXY/Documents/Codigos/Python/P84_85/DadosBasicos/dados_basicos.xlsx'

# Read the Excel file
df_db = pd.read_excel(excel_filename, sheet_name=0)  # sheet_name=0 reads the first sheet
df_db = df_db.fillna('')
# display(df_db)

In [46]:
# Consulta API para listar modelos
models_resp = http_client.get(config['TST']['LITELLM_BASE_URL'] + '/models', headers={
    'api-key': config['TST']['API_KEY_MODELOS_TEXTO'],
    'Authorization': 'Bearer ' + config['TST']['API_KEY_MODELOS_TEXTO']
})
models_resp.raise_for_status()

<Response [200 OK]>

In [47]:
all_models = [m['id'] for m in models_resp.json()['data']]
len(all_models)

43

In [48]:
# Modelos de chat
models_chat = sorted(m for m in all_models if 'embedding' not in m)
models_chat

['claude-3-5-haiku-20241022-v1',
 'claude-3-7-sonnet-20250219',
 'claude-instant-v1',
 'claude-v2',
 'claude-v2.1',
 'claude-v3-haiku',
 'claude-v3-sonnet',
 'claude-v35-sonnet',
 'claude-v35-sonnet-v2',
 'command-light-v14',
 'command-r',
 'command-r-plus',
 'command-v14',
 'dall-e-3',
 'gpt-35-turbo-16k',
 'gpt-4o',
 'gpt-4o-2024-08-06',
 'gpt-4o-mini',
 'llama3-1-405b-instruct-v1',
 'llama3-1-70b-instruct-v1',
 'llama3-1-8b-instruct-v1',
 'llama3-2-11b-instruct-v1',
 'llama3-2-1b-instruct-v1',
 'llama3-2-3b-instruct-v1',
 'llama3-2-90b-instruct-v1',
 'llama3-3-70b-instruct-v1',
 'llama3-70b-instruct',
 'llama3-8b-instruct',
 'mistral-7b-instruct',
 'mistral-large',
 'mistral-small',
 'mixtral-8x7b-instruct',
 'nova-lite-v1',
 'nova-micro-v1',
 'nova-pro-v1',
 'stable-diffusion-xl-v1']

In [49]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, engine, max_response_tokens=500):
    response = client.chat.completions.create(
        model=engine,
        messages=messages,
        # temperature=0.5,
        max_tokens=max_response_tokens,
        # top_p=0.9,
    )
    return response.choices[0].message.content


# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

### Extrair texto do PDF

In [50]:
def extrair_texto_pdf(caminho_pdf):
    """Extrai texto de um arquivo PDF"""
    with open(caminho_pdf, 'rb') as arquivo:
        leitor_pdf = PyPDF2.PdfReader(arquivo)
        texto_total = ''
        
        # Iterando sobre cada página do PDF
        for pagina in leitor_pdf.pages:
            texto_pagina = pagina.extract_text()
            texto_total += texto_pagina + '\n'
        
        return texto_total.strip()
    


In [51]:
def analisar_documento(user_message):
    """Analisa o documento PDF e retorna o texto extraído""" 
    base_system_message = "Você é um assistente especializado em suprimentos para grande projetos de construção de plataformas de petróleo. Você é capaz de responder perguntas sobre suprimentos, sobre os projetos e sobre os fornecedores."
    base_system_message += " Você também é capaz de realizar resumos sobre o status de vários pacotes de equipamentos."
    base_system_message += " Peço para você realizar um resumo em 5 linhas dos principais fatos relevantes e 5 linhas dos pontos de atenção (se houver)."
    base_system_message += " Não há necessidade de preencher as 5 linhas caso não exista informação relevante nos relatórios."
    base_system_message += " O resumo deve ser feito em português. Porém não traduzir para o Português os nomes dos equipamentos e fornecedores."
    system_message = base_system_message.strip()

    # Verifica se o texto de entrada está vazio
    if not user_message or len(user_message.strip()) == 0:
        print("Texto de entrada está vazio.")
        return None

    # Create the list of messages
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

    max_response_tokens = 500
    model = 'claude-v35-sonnet-v2'
    print(model.upper())

    try:
        # Inicializa response antes da chamada
        response = None
        
        response = send_message(
            messages,
            engine=model,
            max_response_tokens=max_response_tokens
        )
        
        # Verifica se a resposta não é None
        if response:
            return response
        else:
            print("Não foi possível gerar resumo.")
            return None

    except Exception as e:
        print(f"Erro ao processar documento: {e}")
        print('-' * 30)
        # Retorna None em caso de exceção
        return None


In [52]:
def processar_pacotes(df_db, caminho_base):
    # Dicionário para armazenar textos dos PDFs
    textos_pdfs = {}

    # Iterar sobre os pacotes no dataframe
    for pacote in df_db['PKG_DESCRIPTION']:
        # Caminho completo da pasta do pacote
        caminho_pacote = os.path.join(caminho_base, pacote)

        # Verificar se a pasta existe
        if os.path.exists(caminho_pacote) and os.path.isdir(caminho_pacote):
            # String para armazenar texto consolidado dos PDFs deste pacote
            texto_consolidado_pacote = ""

            # Iterar sobre arquivos na pasta do pacote
            for arquivo in os.listdir(caminho_pacote):
                # Verificar se é um arquivo PDF
                if arquivo.lower().endswith('.pdf'):
                    caminho_completo_pdf = os.path.join(caminho_pacote, arquivo)
                    print(f"Lendo arquivo: {caminho_completo_pdf}")

                    try:
                        texto_pdf = extrair_texto_pdf(caminho_completo_pdf)
                        # Adiciona o texto do PDF ao texto consolidado, 
                        # com uma linha em branco entre os documentos
                        texto_consolidado_pacote += texto_pdf + "\n\n"
                    except Exception as e:
                        print(f"Erro ao processar {caminho_completo_pdf}: {e}")

            # Remove espaços em branco extras no início e no fim
            texto_consolidado_pacote = texto_consolidado_pacote.strip()

            # Armazena o texto consolidado no dicionário
            textos_pdfs[pacote] = texto_consolidado_pacote

    return textos_pdfs


In [53]:
# Uso
caminho_base = r'C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai_reports\files'
pasta_resumos = r"C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai_reports\resumos"
textos_extraidos = processar_pacotes(df_db, caminho_base)

# Analisar os textos extraídos
for pacote, texto in textos_extraidos.items():
    print("-" * 80)
    print(f"Processando Pacote: {pacote}")
    
    # Verifica se o texto não está vazio
    if not texto or len(texto.strip()) == 0:
        print(f"Texto do pacote {pacote} está vazio. Pulando.")
        continue
    
    resumo = analisar_documento(texto)
    
    if resumo is None:
        print(f"Não foi possível gerar resumo para o pacote {pacote}")
        continue

    print(f"Resumo realizado com sucesso. Tamanho: {len(resumo)} caracteres")
    
    # Salvar resumo em arquivo de texto
    nome_arquivo_resumo = f"{pacote.replace('/', '_').replace('\\', '_')}_resumo.txt"
    caminho_resumo = os.path.join(pasta_resumos, nome_arquivo_resumo)
    
    try:
        with open(caminho_resumo, "w", encoding="utf-8") as f:
            f.write(f"RESUMO DO RELATÓRIO: {pacote}\n")
            f.write("=" * 80 + "\n\n")
            f.write(resumo)
        
        print(f"Resumo salvo em: {caminho_resumo}")
    except Exception as e:
        print(f"Erro ao salvar resumo do pacote {pacote}: {e}")

print(f"\nFim do processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Lendo arquivo: C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai_reports\files\API 610 CENTRIFUGAL PUMPS\4_PDFsam_P84 P85 PB Mech ROE Weekly meeting_19-May-2025.pdf
Lendo arquivo: C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai_reports\files\DECK TROLLEY\14_PDFsam_P84 P85 PB Mech ROE Weekly meeting_19-May-2025.pdf
Lendo arquivo: C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai_reports\files\FLARE SYSTEM\10_PDFsam_P84 P85 PB Mech ROE Weekly meeting_19-May-2025.pdf
Lendo arquivo: C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai_reports\files\FRESH WATER CHLORINATION UNIT\23_PDFsam_P84 P85Weekly meeting_22. May.25_Static.pdf
Lendo arquivo: C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai_reports\files\FRESH WATER CHLORINATION UNIT\FRESH WATER CHLORINATION UNIT.pdf
Lendo arquivo: C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai_reports\files\FRESH WATER CHLORINATION UNIT\P84-85 De Nora Seatrium KBR Action Tracker-21-May-25.pdf
Lendo arquivo: C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai

#### Gerando arquivo Excel com base no resumo dos arquivos txt

In [54]:
import os
import pandas as pd
from datetime import datetime

pasta_resumos = r"C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai_reports\resumos"

# Create empty lists to store data
data = []
current_date = datetime.now().strftime('%Y-%m-%d')

# Iterate through each package in df_db
for pkg in df_db['PKG_DESCRIPTION']:
    filename = os.path.join(pasta_resumos, f"{pkg}_resumo.txt")
    
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            content = f.read()
            
            try:
                # Extract sections
                fatos_start = content.find('Principais fatos relevantes:')
                pontos_start = content.find('Pontos de atenção:')
                
                if fatos_start != -1 and pontos_start != -1:
                    # Get fatos relevantes and split into individual items
                    fatos = content[fatos_start:pontos_start].replace('Principais fatos relevantes:', '').strip()
                    fatos_list = [f.strip() for f in fatos.split('\n') if f.strip()]
                    
                    # Get pontos de atenção and split into individual items
                    pontos = content[pontos_start:].replace('Pontos de atenção:', '').strip()
                    pontos_list = [p.strip() for p in pontos.split('\n') if p.strip()]
                    
                    # Add fatos relevantes
                    for fato in fatos_list:
                        data.append({
                            'PKG_DESCRIPTION': pkg,
                            'DATE': current_date,
                            'Tipo': 'Fato Relevante',
                            'Descricao': fato
                        })
                    
                    # Add pontos de atenção
                    for ponto in pontos_list:
                        data.append({
                            'PKG_DESCRIPTION': pkg,
                            'DATE': current_date,
                            'Tipo': 'Ponto de Atenção',
                            'Descricao': ponto
                        })
                    
            except Exception as e:
                print(f"Error processing {pkg}: {str(e)}")
                
# Create DataFrame
df_notes = pd.DataFrame(data)

# Save to Excel
excel_path = r'C:\Users\ELXY\Documents\Codigos\Python\P84_85\notes.xlsx'
df_notes.to_excel(excel_path, sheet_name='notes', index=False)

print(f"Excel file saved to {excel_path}")

Excel file saved to C:\Users\ELXY\Documents\Codigos\Python\P84_85\notes.xlsx
